# Tutorial 1: Loading, Processing, Visualizing, and Storing Data

This short notebook demostrates how `landlens_db` can be used to load, process, visualize, and store street-view data from local file directories and Mapillary servers.

In [ ]:
from landlens_db.handlers.cloud import Mapillary
from landlens_db.handlers.image import Local
from landlens_db.process.snap import create_bbox, get_osm_lines, snap_to_road_network
from landlens_db.handlers.db import Postgres
from landlens_db.geoclasses.geoimageframe import GeoImageFrame

Before we get started, we will need to load our Mapillary API token and other environmental variables. For simplicity, we will use the `dotenv` library to please install this and create a .env file to follow this tutorial. You will also need to make sure that pandas and geopandas are installed in order to manipulate some of the data required for the tutorial.

In [ ]:
import os
import geopandas as gpd
import glob
import pandas as pd

from dotenv import load_dotenv

load_dotenv()

MLY_TOKEN = os.environ.get("MLY_TOKEN")
LOCAL_IMAGES = os.environ.get("LOCAL_IMAGES")
DOWNLOAD_DIR = os.environ.get("DOWNLOAD_DIR")
DATABASE_URL = os.environ.get("DATABASE_URL")
DB_TABLE = os.environ.get("DB_TABLE")

# Loading Images

`landlens_db` provides two simple ways to load images for the first time which can then be processed and stored for further analysis.

## 1. Loading images from local directory

To load images from a local directory, simply call the `load_images` function while providing the source directory to read from. Currently, only `jpeg` images are supported and it is best to provide the full path to the images.

In [ ]:
local_images = Local.load_images(LOCAL_IMAGES)
local_images

The resulting image is a GeoImageFrame, which is a simple extension of a Pandas GeoDataFrame with a few required column definitions and additional methods for visualization and data verification.

## 2. Loading images from mapillary

`landlens_db` was made to work with Mapillary data and it includes helper functions to make calls to the Mapillary API and download and convert Mapillary data into a format for `landlens_db`.

To use `landlens_db` to fetch data from Mapillary, you first need to initialize a Mapillary connection using your Mapillary Secret Token.

In [ ]:
importer = Mapillary(MLY_TOKEN)

`landlens_db` offers a few functions to filter Mapillary data from their API. However, for more advanced filtering, we recommend that users use the `mapillary-python-sdk` and convert the resulting data into a GeoImageFrame.

Here is an example of how to load data using the `fetch_by_id` method of `landlens_db`:

In [ ]:
image_id = 915374089313107
image = importer.fetch_by_id(image_id)
image

By default, `landlens_db` will download all fields from the Mapillary image endpoint and default to `thumb_1024_url` as the `image_url`, however, you may specify a subset of fields using the `fields` argument and only these fields will be downloaded. Note, you must supply at least the `id`, `geometry`, and one of the image url fields.

For example, using the `fetch_within_bbox` method of `landlens_db`:

In [ ]:
bbox = [139.59,35.865358, 139.719, 35.882781]
start = '2022-03-16'
end = '2022-03-16'
fields = ['id', 'altitude', 'captured_at', 'camera_type', 'thumb_1024_url', 
          'compass_angle', 'computed_compass_angle', 'computed_geometry', 'geometry']

images = importer.fetch_within_bbox(bbox, start_date=start, end_date=end, fields=fields)
images.head()

It is also important to realize that Mapillary image urls are not permanent. So, `landlens_db` offers a method to download Mapillary images and return a new `GeoImageFrame` with the updated the `image_url` to the new location.

In [ ]:
images = images.download_images_to_local(DOWNLOAD_DIR, filename_column='name')
images.head()

## Loading data from arbitrary sources
It is also possible to read from any OGC-recognized vector file format, including ESRI shapefile, geojson, and geopackage, or to create a `GeoImageFrame` in the same manner as a geopandas dataframe by initializing it with data so long as it has a `name`, `image_url`, and `geometry` column.

Data can also be imported from a PostreSQL postGIS enabled database. There is more information below on creating and exporting postgres tables for `landlens_db`.

When reading from postgres, it can be beneficial to load a subset of data. This can be important when the database contains upwards of tens of thousands of images. For this purpose, there are several database utility and query functions to select only a subset of the data in the database.

# Processing Images

Now that we have loaded some data, we can perform some simple processing on the images. Check the documentation for the current processing functions available. Here is an example of how `landlens_db` can be used to snap images to road networks.

First, we need a road network to snap your images to. `landlens_db` also offers a helper function to download road networks from Open Street Map within a given bounding box.

In [ ]:
bbox = images['geometry'].total_bounds
network = get_osm_lines(bbox)

Then, calling the `snap_to_road_network` will snap all points to the closest road network (within the provided threshold distance) and will create a new geometry column in the `GeoImageFrame` falled `snapped_geometry` to represent this new point.

In [ ]:
snap_to_road_network(images, 100, network)

## Snapping to a local road network

It is also possible to load your own road network and snap to this. When doing this, it is important that all the image points are within a reasonable distance from any given road in your network and that the threshold is appropriately set. If you suspect that there are images far from a road, and you do need that image to be snapped to the closest road, then be sure to set a high enough threshold.

In [ ]:
roads_path = 'data/roads/*.shp'
road_files = glob.glob(roads_path)
roads = [gpd.read_file(road) for road in road_files]
network = pd.concat(roads, ignore_index=True)

In [ ]:
snap_to_road_network(images, 100, network, realign_camera=True)

# Visualizing Images

`landlens_db` provides a simple way to visualize its `GeoImageFrames` interactively using Folium. The `map` method of a `GeoImageFrame` will plot all images as markers on a map and will display the image on click along with any metadata set using the `additional_properties` argument as well as markers for any provided additional geometry.

In [ ]:
images.map(
    additional_properties=['altitude', 'camera_type'],
    additional_geometries=[
        {'geometry': 'computed_geometry', 'angle': 'computed_compass_angle', 'label': 'Computed'},
        {'geometry': 'snapped_geometry', 'angle': 'snapped_angle', 'label': 'Snapped'},
    ]
)

# Storing Images

`GeoImageFrame` data can be stored in a variety of formats. Given that it is built on GeoPandas the `GeoDataFrame` class, it will take any geodataframe method to save data. For instance, to save a table as a `geopackage`, we simply call:

In [ ]:
images.to_file('data/images_tutorial.gpkg')

However, in the current version when reading a saved vector format it is important to then initialize the GeoDataFrame as a GeoImageFrame if you want to make use of the features of `landlens_db`. For example:

In [ ]:
images_gdf = gpd.read_file('data/images_tutorial.gpkg')
images = GeoImageFrame(images_gdf)

## Saving to a PostgreSQL Database

`landlens_db` also offers functionality to store data in a PostGIS enabled PostgreSQL database. This is done by extending the `to_postgis` method of GeoPandas. There are some constraints, such as unique image_urls, that are automatically applied when storing data, as well as some data validity checks -- see the documentation for details. 

To save a `GeoImageFrame` to a PostgreSQL table, you will need to first initiate a connection to a PostgreSQL database. You can do this using the `ImageDB` class:

In [ ]:
db_con = Postgres(DATABASE_URL)

This database must already exist and have PostGIS loaded. 

Then, you can save using `to_postgis`:

In [ ]:
local_images.to_postgis(DB_TABLE, db_con.engine, if_exists="replace")

### Updating an Existing Table

When saving to PostgreSQL, you can choose to handle existing tables. `to_postgis` offers the same `fail`, `replace` and `append` methods that GeoPandas offers, however, `append` requires that all data going in will not conflict with any existing data. Instead, it is possible to "upsert" (insert and update) data into existing tables using the `upsert_images` class method of `Image_DB`. You may choose to either update conflicting records or skip them by declaring `"update"` or `"nothing"` in the conflict argument of the function.

In [ ]:
db_con.upsert_images(local_images, DB_TABLE, conflict='update')

### Querying an Existing Table

It is also possible to load and filter data from existing postgres connections. `landlens_db` offers simple filter functions to query and filter tables to provide a subset of the data. This can be important when working with very large datasets. For example, to load all images with an altitude greater than 90:

In [ ]:
high_altitude_images = db_con.table(DB_TABLE).filter(altitude__gt=40).all()

high_alt_map = high_altitude_images.map(
    additional_properties=['altitude', 'camera_type'],
    additional_geometries=[
        {'geometry': 'geometry', 'angle': 'compass_angle', 'label': 'Base Geometry'},
    ])
high_alt_map.save('data/query_map.html')